# Convert triples to Graph
1. Node representing sentence s
2. Node representing extraction e
3. s -- contains --> e
4. e -- e[rel] --> e[subject]
5. e -- object --> e[obj][i]
6. e -- negated --> e[negated]
7. e -- enabler --> e[enabler]
8. e -- attribute to --> e[attribution]


In [58]:
!pip3 install pydot
import json

def read_text(filename):
    raw_text = ''
    with open(filename) as file:
        for line in file:
            raw_text += line
    return raw_text

def write_text(text, filename):
    with open(filename, 'w') as file:
        for line in text:
            file.write(line)
            
def read_json(filename):
    with open(filename) as file:
        data = json.load(file)
    return data

def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

In [59]:
def triples_to_graph(triples):
    # types of nodes
    s_nodes = set([])
    e_nodes = set([])
    entities = set([])

    edges = []
    
    sentence_count = 0
    extraction_count = 0
    for sentence in triples:
        
        sentence_count += 1
        s_node = 's_' + str(sentence_count)
        s_nodes.add(s_node)
        
        for extraction in triples[sentence]:
            
            extraction_count += 1
            e_node = 'e_' + str(extraction_count)
            e_nodes.add(e_node)
            
            # s -- contains --> e
            edges.append((s_node, 'contains', e_node))
            # e -- subject --> e[subject][i]
            for sub in extraction['subject']:
                entities.add(sub)
                edges.append((e_node, 'subject', sub))                
            # e -- e[relation][i] --> e[object][j]
            for rel in extraction['relation']:
                for obj in extraction['object']:
                    entities.add(obj)
                    edges.append((e_node, rel, obj))
            # e -- e[modifiers][i][m_rel] --> e[modifiers][i][m_obj]
            if 'modifiers1' in extraction:
                for modifier in extraction['modifiers1']:
                    for rel in modifier['m_rel']:
                        for obj in modifier['m_obj']:
                            entities.add(obj)
                            edges.append((e_node, rel, obj))
            if 'modifiers2' in extraction:
                for modifier in extraction['modifiers2']:
                    for rel in modifier['m_rel']:
                        for obj in modifier['m_obj']:
                            entities.add(obj)
                            edges.append((e_node, rel, obj))

            # e -- negated --> true
            if 'negated' in extraction and extraction['negated']:
                edges.append((e_node, 'negated', 'True'))
            # e -- enabler --> e[enabler]
            if 'enabler' in extraction and extraction['enabler']:
                edges.append((e_node, 'enabler', extraction['enabler']))
            # e -- attribute to --> e[attribution]
            if 'attribution' in extraction and extraction['attribution']:
                edges.append((e_node, 'attribute to', extraction['attribution']))

    return edges, {'s_nodes': list(s_nodes), 'e_nodes': list(e_nodes), 'entities': list(entities)}

# Plot Knowledge Graph

In [73]:
from pyvis.network import Network
import networkx as nx
import matplotlib.pyplot as plt

def plot_graph(graph, filename):
    nx_graph = nx.DiGraph()
    nodes = {}
    type_count = 0
    node_color = [0]
    nx_graph.add_node('True')
    for type in graph['vertices']:
        type_count += 1
        for node in graph['vertices'][type]:
            nx_graph.add_node(node)
            node_color.append(type_count)
    for edge in graph['edges']:
        nx_graph.add_edge(edge[0], edge[2], {'label': edge[1]})

#     for edge in graph['similar_edges']:
#         nx_graph.add_edge(edge[0], edge[2], color = 'r')
#         edge_labels[(edge[0], edge[2])] = edge[1]


    pos = nx.spring_layout(nx_graph)
    nx.draw(nx_graph, pos, node_color = node_color, with_labels=True, font_size = 10)
    edge_labels = nx.get_edge_attributes(nx_graph,'label')
    nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels = edge_labels, font_size = 10)
    
    figure = plt.gcf()
    figure.set_size_inches(50, 50)
    plt.savefig(filename, dpi=100)

In [74]:
triples = read_json('../data/my_extractions.json')
edges, vertices = triples_to_graph(triples)
print(len(edges), len(vertices['s_nodes']) + len(vertices['e_nodes']) + len(vertices['entities']))
write_json({
    'edges': edges,
    'vertices': vertices
}, '../data/my_graph.json')
plot_graph({
    'edges': edges,
    'vertices': vertices
}, '../data/my_graph.png')

# triples = read_json('../data/ollie_canonicalised_1_triples.json')
# edges, vertices = triples_to_graph(triples)
# plot_graph({
#     'edges': edges,
#     'vertices': vertices
# }, '../data/ollie_graph_2.png')

# triples = read_json('../data/openie5_triples.json')
# edges, vertices = triples_to_graph(triples)
# write_json({
#     'edges': edges,
#     'vertices': vertices
# }, '../data/openie_graph.json')
# plot_graph({
#     'edges': edges,
#     'vertices': vertices
# }, '../data/openie5_graph.png')

204 208


TypeError: add_edge() takes 3 positional arguments but 4 were given